In [1]:
import pandas as pd
import random
import numpy as np
df = pd.read_csv("all_history.csv")
df = df.set_index("Date")
df.head(5)

,A,AA,AAAP,AABB,AAC,AAL,AAMC,AAME,AAN,AAOI,...,ZSTN,ZTR,ZTS,ZULU,ZUMZ,ZVLO,ZVTK,ZWBC,ZYNE,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
01/03/2000,43.76,70.69,24.50,77.49,9.87,18.19,9.26,2.09,24.51,9.96,...,7.52,2.46,29.05,0.06,12.44,26.75,3750000.0,12.75,16.25,1.28
01/04/2000,40.42,71.02,25.00,77.49,9.96,19.33,8.65,2.15,20.62,10.10,...,7.20,2.48,29.06,0.06,12.26,25.00,10000000.0,12.75,19.32,1.31
01/05/2000,37.91,75.11,25.26,77.49,9.78,19.05,8.26,2.20,17.75,10.00,...,7.50,2.49,29.08,0.05,12.05,27.50,7500000.0,30.00,24.54,1.32
01/06/2000,36.46,74.13,25.02,77.49,9.83,19.81,8.39,2.09,17.02,10.00,...,7.55,2.51,29.07,0.05,12.12,25.50,4450000.0,25.00,35.14,1.31
01/07/2000,39.50,73.91,24.62,77.49,9.90,20.27,8.96,2.15,16.88,9.97,...,7.31,2.56,29.98,0.05,11.75,27.34,2750000.0,16.00,29.73,1.29


In [12]:
reset_df = df.reset_index()
date_value = reset_df.loc[reset_df["Date"] == "01/05/2000", "Date"].index[0]

print(date_value)

2


In [14]:
reset_df["Date"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 6004 entries, 0 to 6003
Series name: Date
Non-Null Count  Dtype 
--------------  ----- 
6004 non-null   object
dtypes: object(1)
memory usage: 47.0+ KB


In [5]:
# We will store the period, start date, and portfolios in a class file.  We can then have
# an array of Portfolios to compare.

class Portfolio:
    # instance variables/attributes...
    size = 10 # The initial size or num_stocks in the portfolio
    n_periods = 10 # The number of trading days the portfolio analysis takes place over
    start_date_index = 0
    
    weights = pd.DataFrame({}) # Weights that will be assigned to each of the assets in the portfolio
    
    stdev = 0
    hpr = 0
    hpr_annualized = 0
    risk_adj_return = 0
    alpha = 0
    
    history_df = pd.DataFrame({})
    analysis_df = pd.DataFrame({})
    
    # Constructor
    def __init__(self, size, n_periods, start_date="Random"):
        
        self.size = size
        self.n_periods = n_periods
        
        # If a number is given for start_date, set start_date_index to that number
        if (type(start_date) == int):
            # print("Chose an int index for start_date_index")
            self.start_date_index = start_date
        
        # TODO: Else if a date is given, find that index
        elif (start_date != "Random"):
            # TODO: Validate format and catch errors or re-format properly
            start_date = df.reset_index().loc[reset_df["Date"] == "01/05/2000", "Date"].index[0]
            # print("Found index within df at "+str(start_date))
            
            self.start_date_index = start_date
            
        
        # Else (or if no args are given) - randomize
        else:
            self.randomize_start()
        
        
        # Generate history_df
        self.create_portfolio()
        
        # Assign weights
        self.assignWeights()
        
        print("Initialized new Portfolio instance")
        
    
    # Methods...
    
    # NOTE: Both randomize_ricker and randomize_start need access to the massive CSV file and currently take a pre-made df
    # not as an argument, but as a variable inside them.  This will need to be changed in whatever file I move this to.
    def randomize_ticker(self):
        ticker_index = random.randint(0, len(df.columns) - 1)
        sub_df = df.iloc[:, ticker_index]
        return sub_df.name.upper()
    

    def randomize_start(self):
        """Takes a length parameter that will be added to a random start date and represent the period analyzed"""
        # NOTE: This uses the length of df, which does not exist in the class.  Might be better to define the read_csv here in the class file.
        # Perhaps pass it as an argument in randomize_start and have that param point to the read_csv
        rand_start = random.randint(0, (len(df) - self.n_periods))
        
        self.start_date_index = rand_start
        return self.start_date_index
    
    
    def create_portfolio(self):
    #   df to hold the data from each ticker
        prices_df = pd.DataFrame({})

        start_index = self.start_date_index
        
        i = 0
        while i < self.size:
    #       Run the randomizer and trim values
            ticker = self.randomize_ticker()
            ticker_index = df.columns.get_loc(ticker)
            ticker_df = df.iloc[start_index : start_index + self.n_periods, [ticker_index]]

    #       Clean null values and randomize again if necessary
            ticker_df = ticker_df.dropna()
            if (len(ticker_df) < self.n_periods):
                # Break and decrement i if there are null values in the date range
                # print("\nSeries contains null values.  Choosing a different ticker...")
                i -= 1

            else:
                # Add the data as intended.
                if prices_df.empty:
                    prices_df = ticker_df
                else:
                    prices_df = pd.concat([prices_df, ticker_df], axis=1)
    #                   prices_df = prices_df.merge(prices_df, ticker_df)
    #       
            i += 1
        
        # Load with the percentage change columns
        for item in prices_df.columns:
    #       First, load with values
            prices_df[str(item)+" % chg"] = prices_df[item].pct_change()
        
        # print("\n\nFinal prices_df: ")
        # print(prices_df.head(3))
        
    
        self.history_df = prices_df
        return self.history_df
    
    
#   Assign weights
    def assignWeights(self, method="Random"):
        """Takes a df and assigns weights (defaults to random, but can be either Random or Equal.)
        Returns a weights df with columns corresponding to the columns in the df arg.
        """
        # Dict to store weights as values to column name keys
        df_weights = {}

        # Since the df.columns length will at this point include % change values, weights length should be halved.
        weight_array_length = int(len(self.history_df.columns) / 2)
        
        weights = []
        for weight in range(weight_array_length):
            weights.append(random.randint(0, 10000))
            
        new_weights = []
        i = 0
        for weight in weights:
            new_weights.append(round(weights[i] / sum(weights), 2))
            i += 1
            
        # print("weights array:")
        # print(str(new_weights))
        
        # Check whether there are any errors and make the sum = 100
        if (sum(new_weights) < 1):
            # choose a random index
            randIndex = random.randint(0, len(new_weights) - 1)
            # print(new_weights[randIndex])
            new_weights[randIndex] += (1 - sum(new_weights))
            # print(new_weights)
        
        # Only assign random weights to the columns that don't contain %
        if (method == "Random"):
            # print("self.history_df.columns: "+str(self.history_df.columns))
            for item in self.history_df.columns:
                # print("\nitem: "+str(item))
                if "%" not in item:         
                    # print("% not in item.")
                    # Add to the dictionary
                    # print("df_weights[item] now being set to new_weights[self.history_df.get_loc(item)]: ")
                    # print(str(new_weights[self.history_df.columns.get_loc(item)]))
                    df_weights[str(item)] = new_weights[self.history_df.columns.get_loc(item)]
                
        elif (method == "Equal"):
            # Assign equally
            df[str(item)+" Weight"] = 1 / (len(df.columns) / 2)
            
        else:
            print("Choose weighting method: Random or Equal.")
            
        self.weights = df_weights
        # print(self.weights)
        return self.weights
        

    # Compute portfolio statustics
    def get_portfolio_stats(self):
        """Takes a portfolio price history df and its corresponding weights dict from an assign_weights function,
        and returns the weighted HPR and annualized return for the portfolio"""
        
        hpr_items = []
        stdev_items = []
        
        # Reset index to make using loc practical
        self.history_df = self.history_df.reset_index()
        non_date_columns = self.history_df.iloc[:, 1:]
        
        # Compute total cumulative % change for each asset
        for item in non_date_columns.columns:
            # print("\nitem: "+str(item))
            if ("%" not in item) & ("/" not in item):
                # print("% and / not in item.")
                
                beginning_price = non_date_columns.loc[0 , item]
                # print("Beginning price:")
                # print(beginning_price)
                
                ending_price = non_date_columns.loc[non_date_columns.index[-1], item]
                # print("Ending price:")
                # print(ending_price)
                
                item_hpr = (ending_price - beginning_price) / beginning_price
                # print("HPR for "+str(item)+":")
                # print(item_hpr)
                
                hpr_items.append(item_hpr)
                
            elif ("%" in item):
            # Compute standard deviation of daily % changes for each asset
                col_std = non_date_columns[item].std()
                stdev_items.append(col_std)
                
            
            # Multiply by weights...
            # First for HPR array
            weighted_hpr_items = []
            i = 0
            for item in hpr_items:
                weighted_hpr_items.append(hpr_items[i] * self.weights[non_date_columns.columns[i]])
                i += 1
            # Then for the stdev array
            weighted_stdev_items = []
            i = 0
            for item in stdev_items:
                weighted_stdev_items.append(stdev_items[i] * self.weights[non_date_columns.columns[i]])
                i += 1
                
        # print("weighted hpr_items: "+str(weighted_hpr_items))
        portfolio_hpr = sum(weighted_hpr_items)
        # print(portfolio_hpr)
        
        # print("weighted stdev_items: "+str(weighted_stdev_items))
        portfolio_stdev = sum(weighted_stdev_items)
        # print(portfolio_stdev)
        
        # Adjust by the period analyzed...
        
        # Compute the percentage of a trading year this period represents
        trading_days = 252
        df_days = len(non_date_columns.iloc[:, 0])
        # print("trading days in period analyzed: "+str(df_days))
        pct_of_year = df_days / trading_days
        # print("% of trading year: "+str(pct_of_year))
        
        # Extrapolate for annualized return using appropriate formula
        annualized_hpr = (1 + portfolio_hpr) ** (1 / pct_of_year) - 1
        # print("annualized return: "+str(annualized_hpr))
        
        # Apply a similar process to the standard deviation using the sqrt of period analyzed
        annualized_volatility = portfolio_stdev * np.sqrt(df_days)
        # print("annualized volatility: "+str(annualized_volatility))
        
        # Now compute annualized risk-adjusted return
        risk_adj_return = annualized_hpr / annualized_volatility
        # print("Risk-Adjusted Return: "+str(risk_adj_return))

        # Convert all of this information on the portfolio to its own dataframe
        portfolio_statistics = {
            "num_days": [df_days],
            "period_start": [self.history_df.loc[0, "Date"]],
            "period_end": [self.history_df.loc[self.history_df.index[-1], "Date"]],
            "num_stocks": [int(len(non_date_columns.columns) / 2)],
            "hpr": [portfolio_hpr],
            "annualized_return": [annualized_hpr],
            "stdev": [portfolio_stdev],
            "annualized_stdev": [annualized_volatility],
            "risk_adj_return": [risk_adj_return]
        }
        stats_df = pd.DataFrame(portfolio_statistics)
        self.analysis_df = stats_df
        
        # Output
        #   Write analysis metrics to analysis_df
        return self.analysis_df

    #   TODO: Compare analysis to benchmark for alpha
    def compare_benchmark(self, benchmark="SPY"):
        """Defaults to SPY, compares using the same n_periods and start_date params"""
        bench_df = pd.DataFrame({})
        bench_performance_df = pd.DataFrame({})
        
        # Pull data for SPY
        bench_df[benchmark] = df.loc[self.start_date_index : self.start_date_index + self.n_periods, [benchmark]]
        
        # Load with the percentage change columns
        for item in bench_df.columns:
    #       First, load with values
            bench_df[str(item)+" % chg"] = bench_df[item].pct_change()
            
        beginning_price = bench_df.loc[0 , benchmark]
        # print("Beginning price:")
        # print(beginning_price)
                
        ending_price = bench_df.loc[df.index[-1], benchmark]
        # print("Ending price:")
        # print(ending_price)
        
        bench_hpr = (ending_price - beginning_price) / beginning_price
        # print("HPR for "+str(bench)+":")
        # print(bench_hpr)
        
        bench_performance_df["hpr"] = bench_hpr
        
        # Compute annualized return
        trading_days = 252
        df_days = len(bench_df.iloc[:, 0])
        # print("trading days in period analyzed: "+str(df_days))
        pct_of_year = df_days / trading_days
        # print("% of trading year: "+str(pct_of_year))
        
        # Extrapolate for annualized return using appropriate formula
        annualized_hpr = (1 + bench_hpr) ** (1 / pct_of_year) - 1
        bench_performance_df["annualized_return"] = annualized_hpr
        
        bench_stdev = bench_df[benchmark].std()
        bench_performance_df["stdev"] = bench_stdev
        
        annualized_volatility = bench_stdev * np.sqrt(df_days)
        bench_performance_df["annualized_stdev"] = annualized_volatility
        
        bench_performance_df["risk_adj_return"] = annualized_hpr / annualized_volatility
        
        # Compute metrics and return as a df
        print(bench_performance_df.head())
        
    
    # Write included tickers and weights to a df
    def get_portfolio_comp(self):
        # return a df with columns corresponding to tickers and weights
        return pd.DataFrame(list(self.weights.items()), columns=["Ticker", "Weight"])


In [10]:
# Testbed
stats_df = pd.DataFrame({})
composition_df = pd.DataFrame({})
bench_df = pd.DataFrame({})
comp_df_cols = []
for x in range(2):
    
    portfolio = Portfolio(size=14, n_periods=10, start_date="11/21/2020")
    
    # Rename columns with a suffix based on the iteration
    suffix = "_"+str(x)
    comp_df_cols.append("Ticker"+suffix)
    comp_df_cols.append("Weight"+suffix)

    # Merge with outer join to preserve non-identical weights and tickers for each portfolio
    composition_df = pd.merge(composition_df, portfolio.get_portfolio_comp(), left_index=True, right_index=True, how="outer")
    stats_df = pd.concat([stats_df, portfolio.get_portfolio_stats()])

    # Store the benchmark comparison
    bench_df = pd.merge(bench_df, portfolio.compare_benchmark(), left_index=True, right_index=True, how="inner") 
    
composition_df.columns = comp_df_cols
    
stats_df = stats_df.reset_index(drop=True)
    
stats_df.head(10)
# composition_df.head(100)

Initialized new Portfolio instance


C:\Users\Jonathan\AppData\Local\Temp\ipykernel_18876\2849658317.py:200: RuntimeWarning: invalid value encountered in double_scalars
  item_hpr = (ending_price - beginning_price) / beginning_price


TypeError: cannot do slice indexing on Index with these indexers [2] of type int64

In [4]:
composition_df.head(30)

,Ticker_0,Weight_0,Ticker_1,Weight_1,Ticker_2,Weight_2,Ticker_3,Weight_3,Ticker_4,Weight_4,...,Ticker_15,Weight_15,Ticker_16,Weight_16,Ticker_17,Weight_17,Ticker_18,Weight_18,Ticker_19,Weight_19
0,IMTL,0.46,SEAC,0.15,EBIX,0.35,LOAN,0.27,MKC-V,0.19,...,BIDU,0.08,TALN,0.31,NEE,0.23,PII,0.10,JMTM,0.26
1,FISI,0.51,COKE,0.65,NORD,0.04,IQI,0.22,NSYS,0.07,...,CSL,0.71,GBX,0.31,SDON,0.14,SENEA,0.34,CCM,0.15
2,GVSI,0.03,MCRAA,0.10,MOV,0.39,EZTD,0.26,HARL,0.27,...,NBTB,0.13,TTI,0.12,AQMS,0.39,PXLW,0.24,CHYL,0.44
3,SDON,0.01,VHC,0.10,NATI,0.22,SEDN,0.25,ASBN,0.47,...,KIM,0.08,DGIX,0.26,COBZ,0.24,MCR,0.32,ENBN,0.15
